In [1]:
import PySimpleGUI as sg
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.applications.resnet50 import preprocess_input
import random
model_path= None
file_path=None
image=None
class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
left_column=sg.Column([
    [sg.Text('',key='class_real',text_color='red')],
    [sg.Image(key='image')]
], element_justification='left')# căn sát bên trái
right_column=sg.Column([
    [sg.Text('Classification result:'), sg.Text('   ',key='result'),
    sg.Text('Probability:'), sg.Text('   ',key='prob')],
    [sg.Graph((700,400),(0,0),(10,150),key='graph')]
],element_justification='left')# căn sát bên trái
layout=[
    [sg.Text('Choose your model',key='model_path_title')],
    [sg.Button('Choose model from browse',key='choose')],
    [sg.Text('Select an imagine from your computer or capture from webcam')],
    [sg.Button('Random',key='random'),sg.Button('Browse', key='file'),sg.Button('Capture',key='cam')],
    [sg.Button('Classify',key='classify'), sg.Button('Save',key='save')],
    [left_column,right_column]#Khối hiển thị gồm 2 cột
    ]
window=sg.Window('CNN MNIST FASHION Demo', layout, size=(800,600),resizable=True)
while True:
    event,values=window.read()
    if event==sg.WIN_CLOSED:
        break
    if event=='choose':#Chọn mô đun CNN
        model_path=sg.popup_get_file('Enter the path to the CNN model', title='Load Model')
        
        if model_path is None or not model_path:
            sg.popup_error('No model path entered')
            window['model_path_title'].update('None')
        else:
            try: 
                model=tf.keras.models.load_model(model_path)
                window['model_path_title'].update(model_path)
            except:
                sg.popup_error('Invalid model path or format')
    
    if event =='random':#Chọn ảnh ngẫu nhiên từ tập test fashion-mnist_test.csv
        try:
            test = pd.read_csv('fashion-mnist_test.csv')
            testing_array = np.array(test,dtype = 'float32')
            i = random.randrange(testing_array.shape[0])
            label = int(testing_array[i,0])
            window['class_real'].update(class_names[label],text_color='red')
            image = testing_array[i][1:].reshape(28,28)#hình 2 chiều
            image=cv2.resize(image,(400,400))
            window['image'].update(data=cv2.imencode('.png',image)[1].tobytes())
            window['result'].update('')
            window['prob'].update('')
            window['graph'].erase()
        except Exception as e:
            sg.popup_error(e)
            print(e)
    if event=='file':#Chọn ảnh trong browser
        file_path=sg.popup_get_file('Enter the path to your image', title='Upload Image')
        if file_path is not None and file_path:
            try:
                image=cv2.imread(file_path)#không thể đọc path có dấu
                if image is not None:
                    image=cv2.resize(image,(400,400))
                    window['image'].update(data=cv2.imencode('.png',image)[1].tobytes())
                    window['class_real'].update('')
                    window['result'].update('')
                    window['prob'].update('')
                    window['graph'].erase()
                else: 
                    sg.popup_error('Read image fail!!!')
            except Exception as e:
                sg.popup_error(e)
                print(e)
        else:
            sg.popup_error('No image path entered')
    if event =='cam':#Dùng camera để bắt ảnh 
        cam=cv2.VideoCapture(0)
        ret, frame = cam.read()
        if ret:
            image=cv2.resize(frame,(400,400))
            window['image'].update(data=cv2.imencode('.png',image)[1].tobytes())
            window['class_real'].update('')
            window['result'].update('')
            window['prob'].update('')
            window['graph'].erase()
        cam.release()
    if event =='classify':#Phân loại ảnh
        if model_path is not None:
            try:
                row,col,*channel=image.shape
                if channel !=[]:#có kênh màu
                    input_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                else:#k kênh màu
                    input_image=image
                    
                input_image=cv2.resize(input_image,(28,28))
                
                if channel !=[]:# chuyển các điểm pixel về khoảng [0,255]
                    input_image=preprocess_input(input_image)
                
                input_image /= float(255)
                input_image=input_image.reshape(1,28,28,1)
                if 'transfer' in model_path:# TH dùng model transfer learning cần chuyển về 3 chiều
                    input_image=input_image.repeat(3, axis=-1)
                    
                prediction=model.predict(input_image)#mảng xác suất [[A,B,C]]
                pred_class=np.argmax(prediction)#lấy chỉ số có giá trị xác suất cao
                pred_arr=tf.nn.softmax(prediction)# chuyển về mảng các xác suất
                pred_prob=round((tf.get_static_value(pred_arr[0,pred_class])*100),2)#giữ giá trị xác suất đó
                
                window['result'].update(class_names[pred_class],text_color='red')
                window['prob'].update(f'{pred_prob}%',text_color='red')
                
                graph=window['graph']
                graph.erase()
                for i in range(10):
                    p=round(tf.get_static_value(pred_arr[0,i]) * 100,2)
                    
                    x=i+0.25
                    graph.draw_rectangle((x,0),(x+0.55,p),
                                        fill_color='gray')
                    
                    graph.draw_text(class_names[i], (x+0.25,p+5), color='red')
                    graph.draw_text(str(p)+'%',(x+0.25,p + 9),color='blue')
                graph.Widget.configure(borderwidth=1, relief='solid')#vẽ viền đóng khung biểu đồ
            except Exception as e:
                sg.popup_error(e)
                print(e)
        else:
            sg.popup_error('Model is none')
    if event=='save':#Lưu lại hình ảnh file.png và kết quả phân loại file.txt
        try:
            file_name=sg.popup_get_text('Enter the file name to save', title='Save image and result')
            if file_name is not None:
                cv2.imwrite(file_name+'.png',image)
                with open(file_name+'.txt','w') as f:
                    f.write(f'Class: {window["result"].DisplayText}\n')
                    f.write(f'Probability: {window["prob"].DisplayText}\n')
                sg.popup('Image and result saved successfully') 
        except:
            sg.popup_error('No image selected or captured or invalid file name')
window.close()   

class:  ndarray
shape:  (28, 28)
strides:  (112, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x1dc03c1d850
byteorder:  little
byteswap:  False
type: float32
None
1/1 [==============================] - 0s 205ms/step
class:  ndarray
shape:  (28, 28)
strides:  (112, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x1dc03c1d850
byteorder:  little
byteswap:  False
type: float32
None
1/1 [==============================] - 0s 29ms/step
class:  ndarray
shape:  (28, 28)
strides:  (112, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x1dc03c1d850
byteorder:  little
byteswap:  False
type: float32
None
1/1 [==============================] - 0s 27ms/step
class:  ndarray
shape:  (28, 28)
strides:  (112, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x1dc03c1d850
byteorder:  little
byteswap:  False
type: float32
None
1/1 [==============================] - 0s 26ms/step
cla